<a href="https://colab.research.google.com/github/rym621jp/MazeCreater-Solver/blob/main/MazeCreater%26Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import random
import math
import copy
from enum import Enum

class Maze:
    ### 定数の列挙 ###
    class MazeObjectType(Enum):
        PATH = 0 # 道
        WALL = 1 # 壁
        PLAYER = 2 # プレイヤー
        END = 3 #終端
        GOAL = 4 #ゴール
        PASSED = 5 #通ったとこ
        ROUTE = 6 #最短経路
    
    class Direction(Enum):
        UP = 0 # 上
        DOWN = 1 # 下
        LEFT = 2 # 左
        RIGHT = 3 # 右
    ### 定数（ここまで） ###
    
    # 初期化メソッド
    def __init__(self, width, height):
        if width<5 or width%2==0 or height<5 or height%2==0:
            sys.stderr.write("縦横のそれぞれ５マス以上の奇数で指定してください")
        self._width = width
        self._height = height
    
        self._data = [[0]] # 生成した迷路のデータ
    
        self._start_path = [] # 進行した地点のポイントを保有するリスト
    
        self._is_created = False # 迷路生成が終わっているのかどうか
        
        self._start = [1,1]
        
        self._goal = [0,0]
    
        self._player_point = None # プレイヤーの現在座標
    
        self._path = []   #pathの座標一覧
    
        self._ends = []   #終端座標一覧
    
        self._steps = 0
        
        self._is_goal = False
        
        self.solve_map = None
        
        self.play_map = None
        print("[Maze] initialize")
    
    # 迷路生成
    def create(self):
        obj = self.MazeObjectType
        self._data = [[obj.WALL for width in range(0, self._width)] for height in range(0, self._height)]
        print("[Maze] create maze")
        
    # 穴掘り法で迷路を掘り進める  
    def dig(self, x, y):
        #print("[Maze] dig maze. x = {0}, y = {1}".format(x, y))
        obj = self.MazeObjectType
        direct = self.Direction
        
        self._data[y][x] = obj.PATH
        self._path.append([x,y])
        
        while True:
            directions = []
            if y-2 > 0 and self._data[y-2][x] == obj.WALL:
                directions.append(direct.UP)
            if y+2 < self._height and self._data[y+2][x] == obj.WALL:
                directions.append(direct.DOWN)
            if x-2 > 0 and self._data[y][x-2] == obj.WALL:
                directions.append(direct.LEFT)
            if x+2 < self._width and self._data[y][x+2] == obj.WALL:
                directions.append(direct.RIGHT)
                
            if len(directions) == 0:
                if len(self._start_path)>=1:
                    if self._start_path[-1]==[x,y]:
                        self._ends.append([x,y])
                break
                
            direction = random.choice(directions)
            
            if direction == direct.UP:
                self._data[y-1][x] = obj.PATH
                self._data[y-2][x] = obj.PATH
                self._path.append([y-1,x])
                self._path.append([y-2,x])
                y -= 2
            if direction == direct.DOWN:
                self._data[y+1][x] = obj.PATH
                self._data[y+2][x] = obj.PATH
                self._path.append([y+1,x])
                self._path.append([y+2,x])
                y += 2
            if direction == direct.LEFT:
                self._data[y][x-1] = obj.PATH
                self._data[y][x-2] = obj.PATH
                self._path.append([y,x-1])
                self._path.append([y,x-2])
                x -= 2
            if direction == direct.RIGHT:
                self._data[y][x+1] = obj.PATH
                self._data[y][x+2] = obj.PATH
                self._path.append([y,x+1])
                self._path.append([y,x+2])
                x += 2
                
            #self.draw()
            self._start_path.append([x,y])
            
            
        start_path_len = len(self._start_path)
        if start_path_len >0:
            path = self._start_path.pop(random.randint(0, start_path_len - 1))
            new_x = path[0]
            new_y = path[1]
            
            self.dig(new_x, new_y)
        else:
            self._is_created = True
            
        
    # プレイヤーを配置する
    def set_player(self, x, y):
        self._data[y][x] = self.MazeObjectType.PLAYER
        self._player_point = [x,y]
        print("[Maze] 　set player. x = {0}, y = {1}".format(x, y))
           
    # プレイヤーを移動させる
    def move_player(self, direction):
        obj = self.MazeObjectType
        direct = self.Direction
        goto = []
        pos = self._player_point
        if direction == direct.UP:
            goto = [pos[0], pos[1]-1]
        elif direction == direct.DOWN:
            goto = [pos[0], pos[1]+1]
        elif direction == direct.LEFT:
            goto = [pos[0]-1, pos[1]]
        elif direction == direct.RIGHT:
            goto = [pos[0]+1, pos[1]]
            
        if self.play_map[goto[1]][goto[0]]==obj.PATH or self.play_map[goto[1]][goto[0]]==obj.PASSED:
            self._steps += 1
            self.play_map[pos[1]][pos[0]] = obj.PASSED
            self.play_map[goto[1]][goto[0]] = obj.PLAYER
            self._player_point = [goto[0], goto[1]]
        elif self.play_map[goto[1]][goto[0]]==obj.GOAL:
            self._steps += 1
            self.play_map[pos[1]][pos[0]] = obj.PASSED
            self.play_map[goto[1]][goto[0]] = obj.PLAYER
            self._player_point = [goto[0], goto[1]]
            self._is_goal = True
            print("ゴールに到達しました({}steps)".format(self._steps))
        
        
        print("[Maze] 　move player : {}".format(direction.name))
        
    def set_goal(self):
        available = []
        layers = []
        for pos in self._ends:
            layers.append(pos[1])
        bottom = max(layers)
        for pos in self._ends:
            if pos[1] == bottom:
                available.append(pos)
        goal = random.choice(available)
        self._data[goal[1]][goal[0]] = self.MazeObjectType.GOAL
        self._goal = goal
            
    # 迷路を描画する
    def draw(self, map_):
        print("[Maze] 　draw")
        obj = self.MazeObjectType
        maze_str = ""
        for height in range(0, self._height):
            for width in range(0, self._width):
                if map_[height][width] == obj.WALL:
                    maze_str += "■"
                elif map_[height][width] == obj.PATH:
                    maze_str += "□"
                elif map_[height][width] == obj.PLAYER:
                    maze_str += "▼"
                elif map_[height][width] == obj.END:
                    maze_str += "●"
                elif map_[height][width] == obj.GOAL:
                    maze_str += "Ⓖ"
                elif map_[height][width] == obj.PASSED:
                    maze_str += "✓"
                elif map_[height][width] == obj.ROUTE:
                    maze_str += "☆"
            maze_str += "\n"
        print(maze_str)
        
    def ends_addopt(self):
        obj = self.MazeObjectType
        for pos in self._ends:
            self._data[pos[1]][pos[0]] = obj.END
            
    def detector(self, pos):
        obj = self.MazeObjectType
        result = self._data[pos[1]][pos[0]]
        return result
    
    def solver(self):
        def UP(pos):
            return [pos[0], pos[1]-1]
        def DOWN(pos):
            return [pos[0], pos[1]+1]
        def LEFT(pos):
            return [pos[0]-1, pos[1]]
        def RIGHT(pos):
            return [pos[0]+1, pos[1]]
        obj = self.MazeObjectType
        step_map = [[-1 for width in range(0, self._width)] for height in range(0, self._height)]
        ref = copy.deepcopy(self._data)
        self.solve_map = copy.deepcopy(self._data)
        step_log = [[[1,1]]]
        
        current = self._start
        step_map[current[1]][current[0]] = 0
        step = 0
        goal = False
        goal_pos = []
        while True:
            step += 1
            for current in step_log[step-1]:
                #print(step)
                step_log.append([])
                #print(self.detector(UP(current)),self.detector(DOWN(current)),self.detector(LEFT(current)),self.detector(RIGHT(current)))
                if (self.detector(UP(current))==obj.PATH or self.detector(UP(current))==obj.GOAL) and step_map[UP(current)[1]][UP(current)[0]]==-1:
                    step_map[UP(current)[1]][UP(current)[0]] = step
                    step_log[step].append(UP(current))
                    #print("up")
                    if self.detector(UP(current))==obj.GOAL:
                        #print("goal")
                        goal_pos = UP(current)
                        goal = True
                        break
                if (self.detector(DOWN(current))==obj.PATH or self.detector(DOWN(current))==obj.GOAL) and step_map[DOWN(current)[1]][DOWN(current)[0]]==-1:
                    step_map[DOWN(current)[1]][DOWN(current)[0]] = step
                    step_log[step].append(DOWN(current))
                    #print("down")
                    if self.detector(DOWN(current))==obj.GOAL:
                        #print("goal")
                        goal_pos = DOWN(current)
                        goal = True
                        break
                if (self.detector(LEFT(current))==obj.PATH or self.detector(LEFT(current))==obj.GOAL) and step_map[LEFT(current)[1]][LEFT(current)[0]]==-1:
                    step_map[LEFT(current)[1]][LEFT(current)[0]] = step
                    step_log[step].append(LEFT(current))
                    #print("left")
                    if self.detector(LEFT(current))==obj.GOAL:
                        #print("goal")
                        goal_pos = LEFT(current)
                        goal = True
                        break
                if (self.detector(RIGHT(current))==obj.PATH or self.detector(RIGHT(current))==obj.GOAL) and step_map[RIGHT(current)[1]][RIGHT(current)[0]]==-1:
                    step_map[RIGHT(current)[1]][RIGHT(current)[0]] = step
                    step_log[step].append(RIGHT(current))
                    #print("right")
                    if self.detector(RIGHT(current))==obj.GOAL:
                        #print("goal")
                        goal_pos = RIGHT(current)
                        goal = True
                        break
                                
                #print(step_log[step])
            if goal:
                #print("goal")
                break
        print("最短距離は{}ステップです".format(step))
        #for line in step_map:
            #print(line)
        #print(goal_pos)
        
        c_pos = goal_pos
        for step_ in range(step-1, 0, -1):
            for pos in step_log[step_]:
                if (pos[0]-c_pos[0])**2 + (pos[1]-c_pos[1])**2 == 1:
                    c_pos = pos
                    self.solve_map[c_pos[1]][c_pos[0]] = obj.ROUTE
                    break
                    
    def start_game(self):
        self.create()
        self.dig(1,1)
        self.set_player(1,1)
        self.set_goal()
        self.play_map = copy.deepcopy(self._data)
        maze.draw(self.play_map)
        while True:
            direct = None
            _input = input("wasdで進む方向を指定してください。終了するには「end」を入力してください。")
            if _input == "end":
                break
            elif _input == "w":
                direct = self.Direction.UP
            elif _input == "s":
                direct = self.Direction.DOWN
            elif _input == "a":
                direct = self.Direction.LEFT
            elif _input == "d":
                direct = self.Direction.RIGHT
            else:
                continue
        
            self.move_player(direct)
            self.draw(self.play_map)
            if self._is_goal:
                break
                
    def create_game(self):
        self.create()
        self.dig(1,1)
        self.set_player(1,1)
        self.set_goal()
        maze.draw(self._data)
                
    def solve_game(self):
        self.solver()
        self.draw(self.solve_map)

In [ ]:
width = 13
height = 13
    
maze = Maze(width, height)
maze.start_game()

print("模範解答")
maze.solve_game()